In [0]:
%python

POS_Countries =  spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/christopher_price1@next.co.uk/POS_Countries-1.csv")
SLAs =  spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/christopher_price1@next.co.uk/SLAs_week_13.csv")
WeekDaysList = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/christopher_price1@next.co.uk/Country_Weekends-2.csv")
WeekDaysList.createOrReplaceTempView("WeekDaysList")
POS_Countries.createOrReplaceTempView("POS_Countries")
SLAs.createOrReplaceTempView("SLAs")


In [0]:
%sql
create or replace table Int_KPI_Detail as

Select distinct 

KPI.*,
ROW_NUMBER()over(partition by parcel_barcode order by target_despatch_date desc) as abc

from WhseFin_SharedArea.International_KPIs_Results_2Weeks as KPI
left join imports.fin_dates_with_keys as d on d.date = kpi.target_despatch_date


where d.CurrentWeekCenteredKey = '-2'
and filedateyear in ('2023')

Order by parcel_barcode 

num_affected_rows,num_inserted_rows


In [0]:
%sql
create or replace table International_KPI_Results

Select * from Int_KPI_Detail 
where year(target_despatch_Date) in ('2022','2023') 
and generation_date >= '2022-11-01' 
and abc = '1'

num_affected_rows,num_inserted_rows


In [0]:
%sql
create
or replace table D2D1
select
  R.*,
  case
    when Destination = 'RU' then case
      when substring(r.parcel_barcode, 1, 5) = '64502' then 'DRD RU'
      when substring(r.parcel_barcode, 1, 5) = '64501' then 'Pony Express'
      else null
    end
    when r.Depot = '62' then 'DPD Eire'
    when r.Depot = '51' then 'DHL International'
    when r.Depot = '44' then 'Direct Link'
    when r.Depot = '61' then 'Meest'
    when r.Depot = '63' then 'I Parcel'
    when r.Depot = '74' then 'DHL EU Next Day'
    when r.Depot = '75' then 'DHL EU'
    when r.Depot = '76' then 'Worldnet'
    when r.Depot = '77' then 'SkyNet'
    when r.Depot = '65' then 'DHL Paket'
    when r.Depot = '52' then '3Suisse'
    when r.Depot = '71' then 'EziBuy'
    when r.Depot = '70' then 'DPD International'
    when r.Depot = '33' then 'Hermes'
    when r.Depot = '64' then 'SPSR'
  end as Carrier,
  case
    When Whse = 'Elmsall' then 'UK'
    When Whse = 'Stadium' then 'UK'
    When Whse = 'DVP' then 'UK'
    When Whse = 'Childsplay' then 'UK'
    When Whse = 'Redhouse' then 'UK'
    When Whse = 'VNA' then 'UK'
    When Whse = 'Germany' then 'Hub'
    When Whse = 'Belfast' then 'Hub'
    When Whse = 'Russia' then 'Hub'
    When Whse = 'Reiss' then 'UK'
  end as Warehouse_Type,
  Case
when substring(pr.client_id,1,2) in ('RS') and Warehouse in ('S') then 'Reiss - PSS'
when substring(pr.client_id,1,2) in ('VS') and Warehouse in ('S') then 'VS - PSS'
when substring(pr.client_id,1,2) in ('RS') then 'Reiss'
when substring(pr.client_id,1,2) in ('VS') then 'VS'
when substring(pr.client_id,1,2) in ('GA') then 'GAP'
when substring(pr.client_id,1,2) in ('CD','CP') then 'Childsplay'
when substring(pr.client_id,1,2) in ('AN') then 'Aubin'
when substring(pr.client_id,1,2) in ('LA') then 'Laura Ashley'
else 'Next' end as client




from
  International_KPI_Results as R
  
  left join warehouse.parcel_records as pr on pr.parcel_barcode = r.parcel_barcode and r.account_no = pr.account_no and pr.order_date = r.order_date

num_affected_rows,num_inserted_rows


In [0]:
%sql
create
or replace table D2D2
select
  D.*,
  --case
    --when (
     -- Warehouse_Type = 'UK'
     -- and Destination = 'DE'
   -- ) then array_join(array('UK', string(Depot), Destination), '')
   -- when (
   --   Warehouse_Type = 'UK'
   --   and Order_Type = 'Partner'
   -- ) then array_join(array('UK', string(Depot), Destination), '')
   -- else array_join(array(string(Depot), Destination), '')
 -- end as Lookup,
  case
    when No_Stock_Date != '-' then 'No Stock'
    when (
      case
        When Warehouse_Despatch_Result = 'On Time in Full' then 'Success'
        When Warehouse_Despatch_Result = 'Parcel Despatched Late' then 'Fail'
        When Warehouse_Despatch_Result = 'Full Parcel No Stock' then 'No Stock'
        When Warehouse_Despatch_Result = 'Parcel Incomplete No Stocked Item - Despatched Late' then 'Fail'
        When Warehouse_Despatch_Result = 'Parcel Incomplete No Stocked Item - Despatched On Time' then 'Success'
      end = 'Fail'
      and Target_Despatch_Date = Despatch_Date
    ) then 'Success'
    else case
      When Warehouse_Despatch_Result = 'On Time in Full' then 'Success'
      When Warehouse_Despatch_Result = 'Parcel Despatched Late' then 'Fail'
      When Warehouse_Despatch_Result = 'Full Parcel No Stock' then 'No Stock'
      When Warehouse_Despatch_Result = 'Parcel Incomplete No Stocked Item - Despatched Late' then 'Fail'
      When Warehouse_Despatch_Result = 'Parcel Incomplete No Stocked Item - Despatched On Time' then 'Success'
    end
  end as warehouse_Fails,
  case
  when Destination = 'Otto' and Depot = '33' then '33Otto'
when Destination = 'Zalando DE' and Depot = '65' then '65Zalando DE'
when Destination = 'Zalando NL' and Depot = '65' then '65Zalando NL'
when Destination = 'Zalando FR' and Depot = '76' then '76Zalando FR'
when Destination = 'Zalando NL' and Depot = '76' then '76Zalando NL'
when Destination = 'Amazon DE' and Depot = '65' then '65Amazon DE'
when Destination = 'Amazon FR' and Depot = '76' then '76Amazon FR'
when Destination = 'EziBuy AU' and Depot = '71' then '71EziBuy AU'
when Destination = 'EziBuy NZ' and Depot = '71' then '71EziBuy NZ'
when Destination = 'Spartoo' and Depot = '76' then '76Spartoo'
when Destination = 'Amazon IT' and Depot = '76' then '76Amazon IT'
when Destination = 'Amazon US' and Depot = '51' then '51Amazon US'
when Destination = 'Amazon ES' and Depot = '76' then '76Amazon ES'
when Destination = 'Otto' and Depot = '33' then '33Otto'
when Destination = 'Zalando DE' and Depot = '65' then '65Zalando DE'
when Destination = 'Zalando NL' and Depot = '65' then '65Zalando NL'
when Destination = 'Zalando FR' and Depot = '76' then '76Zalando FR'
when Destination = 'Zalando NL' and Depot = '76' then '76Zalando NL'
when Destination = 'Amazon DE' and Depot = '65' then '65Amazon DE'
when Destination = 'Amazon FR' and Depot = '76' then '76Amazon FR'
when Destination = 'EziBuy AU' and Depot = '71' then '71EziBuy AU'
when Destination = 'EziBuy NZ' and Depot = '71' then '71EziBuy NZ'
when Destination = 'Spartoo' and Depot = '76' then '76Spartoo'
when Destination = 'Amazon IT' and Depot = '76' then '76Amazon IT'
when Destination = 'Amazon US' and Depot = '51' then '51Amazon US'
when Destination = 'Amazon ES' and Depot = '76' then '76Amazon ES'
when Destination = 'La redoute' and Depot = '76' then '76La redoute'
else array_join(array(Client,warehouse_type,depot,destination),'') end as lookup
from
  D2D1 as D

num_affected_rows,num_inserted_rows


In [0]:
%sql
create
or replace table d2d3
select
  d.*,
  case
    when No_Stock_Date != '-' then 'No Stock'
    when warehouse_Fails = 'Success' then '0'
    when warehouse_Fails = 'Fail'
    and Despatch_Date is null then 'Not Yet Despatched'
    when warehouse_Fails = 'Fail'
    and Despatch_Date is not null then datediff(Despatch_Date, Target_Despatch_Date)
  end as Days_Late_Next,
  s.`Total SLA` as SLA,
  case
    when p.`pos country` = 'Yes' then 'Yes'
    when p.`pos country` = 'No' then 'No'
    else 'No'
  end as POS_country,
  case
    when s.pattern = '0' then '1111111'
    when length(s.pattern) = 1 then '1111110'
    when length(s.pattern) = 2 then array_join(
      array(
        '11111',case
          when substring(s.pattern, 1, 1) = '0' then '1'
          when substring(s.pattern, 1, 1) = '1' then '0'
        end,
        case
          when substring(s.pattern, 2, 1) = '0' then '1'
          when substring(s.pattern, 2, 1) = '1' then '0'
        end
      ),
      ''
    )
    when length(s.pattern) = 3 then array_join(
      array(
        '1111',case
          when substring(s.pattern, 1, 1) = '0' then '1'
          when substring(s.pattern, 1, 1) = '1' then '0'
        end,
        case
          when substring(s.pattern, 2, 1) = '0' then '1'
          when substring(s.pattern, 2, 1) = '1' then '0'
        end,case
          when substring(s.pattern, 3, 1) = '0' then '1'
          when substring(s.pattern, 3, 1) = '1' then '0'
        end
      ),
      ''
    )
    when length(s.pattern) = 4 then array_join(
      array(
        '111',case
          when substring(s.pattern, 1, 1) = '0' then '1'
          when substring(s.pattern, 1, 1) = '1' then '0'
        end,
        case
          when substring(s.pattern, 2, 1) = '0' then '1'
          when substring(s.pattern, 2, 1) = '1' then '0'
        end,case
          when substring(s.pattern, 3, 1) = '0' then '1'
          when substring(s.pattern, 3, 1) = '1' then '0'
        end,case
          when substring(s.pattern, 4, 1) = '0' then '1'
          when substring(s.pattern, 4, 1) = '1' then '0'
        end
      ),
      ''
    )
    when length(s.pattern) = 5 then array_join(
      array(
        '11',case
          when substring(s.pattern, 1, 1) = '0' then '1'
          when substring(s.pattern, 1, 1) = '1' then '0'
        end,
        case
          when substring(s.pattern, 2, 1) = '0' then '1'
          when substring(s.pattern, 2, 1) = '1' then '0'
        end,case
          when substring(s.pattern, 3, 1) = '0' then '1'
          when substring(s.pattern, 3, 1) = '1' then '0'
        end,case
          when substring(s.pattern, 4, 1) = '0' then '1'
          when substring(s.pattern, 4, 1) = '1' then '0'
        end,case
          when substring(s.pattern, 5, 1) = '0' then '1'
          when substring(s.pattern, 5, 1) = '1' then '0'
        end
      ),
      ''
    )
    when length(s.pattern) = 6 then array_join(
      array(
        '1',case
          when substring(s.pattern, 1, 1) = '0' then '1'
          when substring(s.pattern, 1, 1) = '1' then '0'
        end,
        case
          when substring(s.pattern, 2, 1) = '0' then '1'
          when substring(s.pattern, 2, 1) = '1' then '0'
        end,case
          when substring(s.pattern, 3, 1) = '0' then '1'
          when substring(s.pattern, 3, 1) = '1' then '0'
        end,case
          when substring(s.pattern, 4, 1) = '0' then '1'
          when substring(s.pattern, 4, 1) = '1' then '0'
        end,case
          when substring(s.pattern, 5, 1) = '0' then '1'
          when substring(s.pattern, 5, 1) = '1' then '0'
        end,case
          when substring(s.pattern, 6, 1) = '0' then '1'
          when substring(s.pattern, 6, 1) = '1' then '0'
        end
      ),
      ''
    )
    when length(s.pattern) = 7 then array_join(
      array(
        case
          when substring(s.pattern, 1, 1) = '0' then '1'
          when substring(s.pattern, 1, 1) = '1' then '0'
        end,
        case
          when substring(s.pattern, 2, 1) = '0' then '1'
          when substring(s.pattern, 2, 1) = '1' then '0'
        end,case
          when substring(s.pattern, 3, 1) = '0' then '1'
          when substring(s.pattern, 3, 1) = '1' then '0'
        end,case
          when substring(s.pattern, 4, 1) = '0' then '1'
          when substring(s.pattern, 4, 1) = '1' then '0'
        end,case
          when substring(s.pattern, 5, 1) = '0' then '1'
          when substring(s.pattern, 5, 1) = '1' then '0'
        end,case
          when substring(s.pattern, 6, 1) = '0' then '1'
          when substring(s.pattern, 6, 1) = '1' then '0'
        end,case
          when substring(s.pattern, 6, 1) = '0' then '1'
          when substring(s.pattern, 6, 1) = '1' then '0'
        end
      ),
      ''
    )
  end as PythonPattern
  
from
  d2d2 as d
  left join SLAs as s on d.lookup = s.concat2
  left join POS_Countries as p on p.code = d.Destination

num_affected_rows,num_inserted_rows


In [0]:
%sql
create
or replace table to_python
select
  *,
  case
    when (promise_date is not null and Generation_date is not null ) and datediff(promise_date, Generation_date) < '4' and Days_late_next in ('No Stock','Not Yet Despatched') then 1
    when Days_late_next = 'No Stock' then 0
    when Days_late_next = 'Not Yet Despatched' then 0
    when Days_Late_Next = 'Fail' then 0
    when Days_Late_Next is null then 0
    else Days_Late_Next
  end as Days_Late_Next_temp,
   
    
  case when PythonPattern is null then '1111100'  else PythonPattern end as PythonPatternNew
FROM
  d2d3
  
  where  sla is not null 

num_affected_rows,num_inserted_rows


In [0]:
%sql
create or replace table to_python2

select 
*,
case
when datediff(promise_date, Generation_date) < '4' then date_add(Target_despatch_date,int(array_min(array(Days_Late_Next_temp,1))))
when promise_date is not null then promise_date
when datediff(promise_date, Generation_date) >= '4' then date_add(Target_despatch_date,int(array_max(array(Days_Late_Next_temp,1))))
else date_add(Target_despatch_date,int(array_max(array(Days_Late_Next_temp,1))))
end as Target_despatch_date_temp

from to_python

num_affected_rows,num_inserted_rows


In [0]:
%sql
create or replace table null_SLA
  
select
*,
case
    when Days_late_next = 'No Stock' then 0
    when Days_late_next = 'Not Yet Despatched' then 0
    when Days_Late_Next = 'Fail' then 0
    when Days_Late_Next is null then 0
    else Days_Late_Next
  end as Days_Late_Next_temp,
  case when PythonPattern is null then '1111100' else PythonPattern end as PythonPatternNew
  

from d2d3 where sla is null 

num_affected_rows,num_inserted_rows


In [0]:
%sql
create or replace table null_sla2



select 
*,
case
when datediff(promise_date, Generation_date) < '4' then date_add(Target_despatch_date,int(array_min(array(Days_Late_Next_temp,1))))
when promise_date is not null then promise_date
when datediff(promise_date, Generation_date) >= '4' then date_add(Target_despatch_date,int(array_max(array(Days_Late_Next_temp,1))))
else date_add(Target_despatch_date,int(array_max(array(Days_Late_Next_temp,1))))
end as Target_despatch_date_temp,
'#N/A' as Busday,
'#N/A' as Delivery_Cut_Off_Temp

from null_sla

num_affected_rows,num_inserted_rows


In [0]:
%pip install koalas

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
%python
import numpy as np
import pyspark.pandas as ps
from datetime import datetime, timedelta

import databricks.koalas as ks
from datetime import datetime

Int_parcels = spark.sql("SELECT *  FROM to_python2 where sla is not null and PythonPattern is not null")
Int_parcels = Int_parcels.to_koalas()




In [0]:
%python

Int_parcels['Promise_Date'] = ks.to_datetime(Int_parcels['Promise_Date'])
Int_parcels['Generation_Date'] = ks.to_datetime(Int_parcels['Generation_Date'])
Int_parcels['Target_Despatch_Date'] = ks.to_datetime(Int_parcels['Target_Despatch_Date'])
Int_parcels['Target_despatch_date_temp'] = ks.to_datetime(Int_parcels['Target_despatch_date_temp'])
Int_parcels['Days_Late_Next_temp'] = ks.to_numeric(Int_parcels['Days_Late_Next_temp'])
Int_parcels['SLA'] = ks.to_numeric(Int_parcels['SLA'])



/databricks/spark/python/pyspark/sql/pandas/functions.py:396: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


In [0]:
%python

ks.set_option('compute.ops_on_diff_frames', True)

Int_parcels['Busday'] = Int_parcels.apply(lambda row:   np.is_busday(row.Target_despatch_date_temp.date(), weekmask = row.PythonPatternNew), axis =  1)

In [0]:
%python

Int_parcels['Delivery_Cut_Off_Temp'] = Int_parcels.apply(lambda row:   np.busday_offset(row.Target_despatch_date_temp.date(), row.SLA, weekmask = str(row.PythonPatternNew),roll="forward") if row.Busday == True else np.busday_offset(row.Target_despatch_date_temp.date(), row.SLA - 1, weekmask = str(row.PythonPatternNew),roll="forward"), axis =  1)


Int_parcels['Delivery_Cut_Off_Temp']=Int_parcels['Delivery_Cut_Off_Temp'].astype(str)
Int_parcels['Busday']=Int_parcels['Busday'].astype(str)



In [0]:
%sql
drop table post_python

In [0]:
%python 

Int_parcels.to_table(name = 'post_python',mode = 'overwrite')


In [0]:
%sql
create or replace table all_data 

select * from post_python

union all

select * from null_sla2

num_affected_rows,num_inserted_rows


In [0]:
%sql
create
or replace table d2d4
select
  d.*,
  case
    when datediff(promise_date, generation_date) < 4 then Delivery_Cut_Off_Temp
    when Promise_Date is not null then Promise_Date
    else Delivery_Cut_Off_Temp
  end as Delivery_Cut_Off,
  case
    when promise_date is not null then promise_date
    else case
      when datediff(promise_date, generation_date) < 4 then Delivery_Cut_Off_Temp
      when Promise_Date is not null then Promise_Date
      else Delivery_Cut_Off_Temp
    end
  end as Customer_promise
from
  all_data as d

num_affected_rows,num_inserted_rows


In [0]:
%sql select * from d2d4 where parcel_barcode = '7050023761351062'

Target_Despatch_Week,Generation_Date,Target_Despatch_Date,Parcel_Barcode,Parcel_airwaybill,Account_No,Order_Date,Promise_Date,Order_Type,Whse,MOD,Batch,Items_Despatched,Total_Items,Depot,Destination,Packing_Station,Warehouse_Despatch_Result,Despatch_Date,No_Stock_Date,Russia_Refused,Latest_Status_Description,Latest_Status_Date,Latest_Status_Time,Delivered_Description,Delivery_Date,Delivery_Time,Delivered_Code,Delivered_Sub_Code,FileDateTime,FileDateYear,FileDateDay,FileDateMonth,abc,Carrier,Warehouse_Type,client,warehouse_Fails,lookup,Days_Late_Next,SLA,POS_country,PythonPattern,Days_Late_Next_temp,PythonPatternNew,Target_despatch_date_temp,Busday,Delivery_Cut_Off_Temp,Delivery_Cut_Off,Customer_promise


In [0]:
%sql
create
or replace table d2d5
select
  d.*,
  case
    when (delivery_date is not null and customer_promise is not null) and delivery_date <= Delivery_Cut_Off then 'Success'
    when (delivery_date is not null and customer_promise is not null) and delivery_date > Delivery_Cut_Off then 'Fail'
    when promise_date is not null
    and delivery_date > promise_date then 'Fail'
    when delivery_date is null then 'Fail'
    else 'Success'
  end as Customer_result,
  case
    when promise_date is null then Delivery_Cut_Off
    when POS_Country = 'Yes' then promise_date
    else Delivery_Cut_Off
  end as Max_Delivery_date
from
  d2d4 as d

num_affected_rows,num_inserted_rows


In [0]:
%sql
create
or replace table d2d6
select
  d.*,
  case
    when (
      target_despatch_date is null
      and delivery_date is not null
    ) then 'Success'
    when No_Stock_date != '-' then 'No Stock'
    when delivery_date is not null and Delivery_Cut_Off is not null and delivery_date <= Delivery_Cut_Off then 'Success' --
    when (
      Customer_result = 'Fail'
      and warehouse_fails = 'Success'
    ) then 'Fail'
    when case
      when delivery_date is not null and Delivery_Cut_Off is not null and delivery_date > Delivery_Cut_Off then 'Fail'
      when delivery_date is not null and Delivery_Cut_Off is not null and delivery_date <= Delivery_Cut_Off then 'Success'
      else null
    end is not null then case
      when delivery_date is not null and Delivery_Cut_Off is not null and delivery_date > Delivery_Cut_Off then 'Fail'
      when delivery_date is not null and Delivery_Cut_Off is not null and delivery_date <= Delivery_Cut_Off then 'Success'
    end
    else 'Fail'
  end as carrier_result,
  case
    when datediff(customer_promise, generation_date) <= 3 then 'Fail'
    when warehouse_fails = 'Success' then warehouse_fails
    when warehouse_fails = 'No Stock' then warehouse_fails
    when warehouse_fails = 'Fail'
    and customer_result = 'Success' then 'Success'
    else warehouse_fails
  end as Next_Fails
from
  d2d5 as d

num_affected_rows,num_inserted_rows


In [0]:
%sql
create
or replace table d2d7
select
  d.*,
  case
    when carrier_result = 'No Stock' then 'No Stock'
    when Customer_result = 'Success' then '0'
    when Customer_result = 'Fail'
    and Delivery_date is null then 'Not Delivered'
    when Customer_result = 'Fail'
    and Delivery_date is not null then case
      when Customer_result = 'Fail' then case
        when datediff(delivery_date, customer_promise) < 0 then datediff(delivery_date, promise_date)
        else datediff(delivery_date, customer_promise)
      end
      when Customer_result != 'Fail' then '0'
    end
  end as Days_late,
  substring(d.parcel_barcode, 1, 5) as barcode_substring,
  case
    when Max_Delivery_date >= current_date() then 'Exclude'
    else 'Include'
  end as Due_for_delivery,
  case
    when carrier_result = 'No Stock' then carrier_result
    when Customer_result = 'Success' then Customer_result
    when Next_fails != 'Success' then 'Next Fail'
    else 'Carrier Fail'
  end as POT,
  
  
  
  case
    when carrier_result = 'No Stock' then 'No Stock'
    else case
      when case
        when carrier_result = 'Success' then 0
        else array_max(
          array(
            datediff(delivery_date, delivery_cut_off),case
              when promise_date is null then null
              else datediff(delivery_date, delivery_cut_off)
            end
          )
        )
      end is null then 'Not Delivered'
      else case
        when carrier_result = 'Success' then 0
        else array_max(
          array(
            datediff(delivery_date, delivery_cut_off),case
              when promise_date is null then null
              else datediff(delivery_date, delivery_cut_off)
            end
          )
        )
      end
    end
  end as Carrier_Days_Late,
  Case
when substring(pr.client_id,1,2) in ('RS') and Warehouse in ('S') then 'Reiss - PSS'
when substring(pr.client_id,1,2) in ('VS') and Warehouse in ('S') then 'VS - PSS'
when substring(pr.client_id,1,2) in ('RS') then 'Reiss'
when substring(pr.client_id,1,2) in ('VS') then 'VS'
when substring(pr.client_id,1,2) in ('GA') then 'GAP'
when substring(pr.client_id,1,2) in ('CD','CP') then 'Childsplay'
when substring(pr.client_id,1,2) in ('AN') then 'Aubin'
when substring(pr.client_id,1,2) in ('LA') then 'Laura Ashley'
else 'Next' end as client2



from
  d2d6 as d
  left join warehouse.parcel_records as pr on pr.parcel_barcode = d.parcel_barcode and d.account_no = pr.account_no and pr.order_date = d.order_date

num_affected_rows,num_inserted_rows


In [0]:
%sql
create or replace table KPI_data

select
Target_Despatch_Week,
date(Generation_Date) as Generation_Date,
date(Target_Despatch_Date) as Target_Despatch_Date,
Parcel_Barcode,
Parcel_airwaybill,
Account_No,
date(Order_Date) as Order_Date,
date(Promise_Date) as Promise_Date,
Order_Type,
Whse,
MOD,
Batch,
Items_Despatched,
Total_Items,
Depot,
Destination,
Packing_Station,
Warehouse_Despatch_Result,
date(Despatch_Date) as Despatch_Date,
date(No_Stock_Date) as No_Stock_Date,
Russia_Refused,
Latest_Status_Description,
date(Latest_Status_Date) as Latest_Status_Date,
Latest_Status_Time,
Delivered_Description,
date(Delivery_Date) as Delivery_Date,
Delivery_Time,
Delivered_Code,
Delivered_Sub_Code,
FileDateTime,
FileDateYear,
FileDateDay,
FileDateMonth,
Carrier,
Lookup,
Warehouse_Type,
Warehouse_Fails,
Days_Late_Next,
Next_Fails,
SLA,
date(Customer_Promise) as Customer_Promise,
date(Delivery_Cut_Off) as Delivery_Cut_Off,
Carrier_Result,
Carrier_Days_Late,
Customer_Result,
Days_Late,
Barcode_Substring,
'NA' as Russia_carrier,
'NA' as Russia_Warehouse,
date(Max_Delivery_Date) as Max_Delivery_Date,
Due_for_Delivery,
POS_Country,
Pot,
client2 as Client

from d2d7


order by parcel_barcode desc



num_affected_rows,num_inserted_rows


#KPI Results

In [0]:
%sql
--select * from KPI_data where year(Generation_Date) = 2023

In [0]:
%python

Days_To_Deliver = spark.sql("SELECT *  FROM d2d7 where sla is not null and PythonPattern is not null and Delivery_Date is not null")
Days_To_Deliver = Days_To_Deliver.to_koalas()

Days_To_Deliver['Promise_Date'] = ks.to_datetime(Days_To_Deliver['Promise_Date'])
Days_To_Deliver['Delivery_Date'] = ks.to_datetime(Days_To_Deliver['Delivery_Date'])
Days_To_Deliver['Generation_Date'] = ks.to_datetime(Days_To_Deliver['Generation_Date'])
Days_To_Deliver['Target_Despatch_Date'] = ks.to_datetime(Days_To_Deliver['Target_Despatch_Date'])
Days_To_Deliver['Customer_promise'] = ks.to_datetime(Days_To_Deliver['Customer_promise'])

Days_To_Deliver['TargetDespatch_Busday'] = Days_To_Deliver.apply(lambda row:   np.is_busday(row.Target_Despatch_Date.date(),weekmask = row.PythonPatternNew), axis =  1)
Days_To_Deliver['DaysToDeliver_FromTargetDespatch'] = Days_To_Deliver.apply(lambda row:  np.busday_count(row.Target_Despatch_Date.date(), row.Delivery_Date.date(), weekmask = str(row.PythonPatternNew)) \
                                                                            if row.TargetDespatch_Busday == True \
                                                                            else  np.busday_count(row.Target_Despatch_Date.date(), row.Delivery_Date.date(), weekmask = str(row.PythonPatternNew)) + 1, axis=1)

Days_To_Deliver['CustomerPromise_Busday'] = Days_To_Deliver.apply(lambda row:   np.is_busday(row.Customer_promise.date(),weekmask = row.PythonPatternNew), axis =  1)
Days_To_Deliver['DaysToDeliver_FromPromise'] = Days_To_Deliver.apply(lambda row:  np.busday_count(row.Customer_promise.date(), row.Delivery_Date.date(), weekmask = str(row.PythonPatternNew)) if row.CustomerPromise_Busday == True else  np.busday_count(row.Customer_promise.date(), row.Delivery_Date.date(), weekmask = str(row.PythonPatternNew)) + 1, axis=1)

Days_To_Deliver['TargetDespatch_Busday']=Days_To_Deliver['TargetDespatch_Busday'].astype(str)
Days_To_Deliver['DaysToDeliver_FromTargetDespatch']=Days_To_Deliver['DaysToDeliver_FromTargetDespatch'].astype(str)

Days_To_Deliver['CustomerPromise_Busday']=Days_To_Deliver['CustomerPromise_Busday'].astype(str)
Days_To_Deliver['DaysToDeliver_FromPromise']=Days_To_Deliver['DaysToDeliver_FromPromise'].astype(str)


/databricks/spark/python/pyspark/sql/pandas/functions.py:396: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


In [0]:
%python
def test(date1,date2,pattern):
  return np.busday_count(datetime.strptime(date1, "%Y-%m-%d").date(), datetime.strptime(date2, "%Y-%m-%d").date(), weekmask = pattern)

In [0]:
%python
print(test("2022-12-02","2022-12-07","1111100"))

In [0]:
%sql
drop table Days_To_Deliver

In [0]:
%python 

Days_To_Deliver.to_table(name = 'Days_To_Deliver',mode = 'overwrite')

In [0]:
%sql
create or replace table Not_delievered

SELECT 
*,
'null SLA or Delivery Date' as TargetDespatch_Busday,
'Not Delivered' as DaysToDeliver_FromTargetDespatch,
'null SLA or Delivery Date' as CustomerPromise_Busday,
'Not Delivered' as DaysToDeliver_FromPromise
FROM d2d7 where sla is  null or PythonPattern is  null or Delivery_Date is  null

In [0]:
%sql
create or replace table D2D_Parcel_data

select * from Not_delievered

union all

select * from Days_To_Deliver

In [0]:
%sql
select * from pre_pivot where customer_result = 'Success' and pot != 'Success'

In [0]:
%sql
create or replace table pre_pivot

select 
*,
case 
when DaysToDeliver_FromTargetDespatch = 'Not Delivered' then POT
when DaysToDeliver_FromTargetDespatch < '0' then '1'
when DaysToDeliver_FromTargetDespatch = '0' then '1'
when DaysToDeliver_FromTargetDespatch ='1' then '1'
when DaysToDeliver_FromTargetDespatch ='2' then '2'
when DaysToDeliver_FromTargetDespatch ='3' then '3'
when DaysToDeliver_FromTargetDespatch ='4' then '4'
when DaysToDeliver_FromTargetDespatch ='5' then '5'
when DaysToDeliver_FromTargetDespatch ='6' then '6'
when DaysToDeliver_FromTargetDespatch ='7' then '7'
when DaysToDeliver_FromTargetDespatch ='8' then '8'
when DaysToDeliver_FromTargetDespatch ='9' then '9'
when DaysToDeliver_FromTargetDespatch ='10' then '10'
when DaysToDeliver_FromTargetDespatch ='11' then '11'
when DaysToDeliver_FromTargetDespatch ='12' then '12'
when DaysToDeliver_FromTargetDespatch ='13' then '13'
when DaysToDeliver_FromTargetDespatch ='14' then '14'
when DaysToDeliver_FromTargetDespatch ='15' then '15'
when DaysToDeliver_FromTargetDespatch >='16' then '16+' 
else DaysToDeliver_FromTargetDespatch end as NewDaysToDeliver_FromDTargetDespatch,

case 
when DaysToDeliver_FromPromise = 'Not Delivered' then POT
when DaysToDeliver_FromPromise < '0' then '1'
when DaysToDeliver_FromPromise = '0' then '1'
when DaysToDeliver_FromPromise ='1' then '1'
when DaysToDeliver_FromPromise ='2' then '2'
when DaysToDeliver_FromPromise ='3' then '3'
when DaysToDeliver_FromPromise ='4' then '4'
when DaysToDeliver_FromPromise ='5' then '5'
when DaysToDeliver_FromPromise ='6' then '6'
when DaysToDeliver_FromPromise ='7' then '7'
when DaysToDeliver_FromPromise ='8' then '8'
when DaysToDeliver_FromPromise ='9' then '9'
when DaysToDeliver_FromPromise ='10' then '10'
when DaysToDeliver_FromPromise ='11' then '11'
when DaysToDeliver_FromPromise ='12' then '12'
when DaysToDeliver_FromPromise ='13' then '13'
when DaysToDeliver_FromPromise ='14' then '14'
when DaysToDeliver_FromPromise ='15' then '15'
when DaysToDeliver_FromPromise >='16' then '16+' 
else DaysToDeliver_FromPromise end as NewDaysToDeliver_FromPromise

from D2D_Parcel_data

In [0]:
%sql
select distinct NewDaysToDeliver_FromDTargetDespatch from pre_pivot

In [0]:
%sql
select * from pre_pivot where NewDaysToDeliver_FromDTargetDespatch = 'Success' and sla is not null

In [0]:
%sql
select count(*) from d2d7

In [0]:
%sql
create or replace table D2D_Pivot

SELECT 
Warehouse_Type,
Due_for_Delivery,
Client,
initcap(country_desc) as country_desc,
COALESCE(`1`, 0) as `1`,
COALESCE(`2`, 0) as `2`,
COALESCE(`3`, 0) as `3`,
COALESCE(`4`, 0) as `4`,
COALESCE(`5`, 0) as `5`,
COALESCE(`6`, 0) as `6`,
COALESCE(`7`, 0) as `7`,
COALESCE(`8`, 0) as `8`,
COALESCE(`9`, 0) as `9`,
COALESCE(`10`, 0) as `10`,
COALESCE(`11`, 0) as `11`,
COALESCE(`12`, 0) as `12`,
COALESCE(`13`, 0) as `13`,
COALESCE(`14`, 0) as `14`,
COALESCE(`15`, 0) as `15`,
COALESCE(`16+`, 0) as `16+`,
COALESCE(`Next Fail`, 0) as Next_Fail,
COALESCE(`Carrier Fail`, 0) as  Carrier_Fail,
COALESCE(`No Stock`, 0) as NoStock,
COALESCE(`Next Fail`, 0) +
COALESCE(`Carrier Fail`, 0) +
COALESCE(`No Stock`, 0) as Total_Undelivered,



COALESCE(`1`, 0) +
COALESCE(`2`, 0) +
COALESCE(`3`, 0) +
COALESCE(`4`, 0) +
COALESCE(`5`, 0) +
COALESCE(`6`, 0) +
COALESCE(`7`, 0) +
COALESCE(`8`, 0) +
COALESCE(`9`, 0) +
COALESCE(`10`, 0) +
COALESCE(`11`, 0) +
COALESCE(`12`, 0) +
COALESCE(`13`, 0) +
COALESCE(`14`, 0) +
COALESCE(`15`, 0) +
COALESCE(`Next Fail`, 0) +
COALESCE(`Carrier Fail`, 0) +
COALESCE(`No Stock`, 0)  +
COALESCE(`16+`, 0) as Total










FROM 

(
SELECT 
parcel_barcode, 
Due_for_Delivery,
Warehouse_Type,
Client,
initcap(trim(dg.country_desc))  as country_desc,
NewDaysToDeliver_FromDTargetDespatch

FROM pre_pivot as pr
left join next_dss.dg_country as dg on pr.Destination = dg.country_code
)
PIVOT (
  count(Parcel_Barcode)
FOR NewDaysToDeliver_FromDTargetDespatch in ('1',
'2',
'3',
'4',
'5',
'6',
'7',
'8',
'9',
'10',
'11',
'12',
'13',
'14',
'15',
'16+',
'Next Fail',
'Carrier Fail',
'No Stock'

  ));


In [0]:
%sql
select * from D2D_Pivot

In [0]:
%sql
select * from D2D_Pivot where country_desc is not null and Due_for_Delivery = 'Include' and client = 'Reiss' order by total desc

In [0]:
%sql
select * from D2D_Pivot where country_desc is not null and Due_for_Delivery = 'Include' and client = 'Next' and Warehouse_Type = 'UK' order by total desc

In [0]:
%sql
select * from D2D_Pivot where country_desc is not null and Due_for_Delivery = 'Include' and client = 'GAP' order by total desc

In [0]:
%sql
select * from D2D_Pivot where country_desc is not null and Due_for_Delivery = 'Include' and client = 'Childsplay' order by total desc

In [0]:
%sql
select * from D2D_Pivot where country_desc is not null and Due_for_Delivery = 'Include' and client = 'Aubin' order by total desc

In [0]:
%sql
select * from D2D_Pivot where country_desc is not null and Due_for_Delivery = 'Include' and client = 'Laura Ashley' order by total desc

In [0]:
%sql
select * from D2D_Pivot where country_desc is not null and Due_for_Delivery = 'Include' and client = 'VS' order by total desc

In [0]:
%sql
select * from D2D_Pivot where country_desc is not null and Due_for_Delivery = 'Include' and client = 'VS - PSS' order by total desc

In [0]:
%sql
select * from D2D_Pivot where country_desc is not null and Due_for_Delivery = 'Include' and client = 'Reiss - PSS' order by total desc

In [0]:
%sql
select 
Due_for_Delivery,
country_desc,
sum(`1`) as `1`,sum(`2`) as `2`,sum(`3`) as `3`,sum(`4`) as `4`,sum(`5`) as `5`,sum(`6`) as `6`,sum(`7`) as `7`,sum(`8`) as `8`,sum(`9`) as `9`,sum(`10`) as `10`,sum(`11`) as `11`,sum(`12`) as `12`,sum(`13`) as `13`,sum(`14`) as `14`,sum(`15`) as `15`,sum(`16+`) as `16+`,
sum(Next_Fail) as Next_Fail,
sum(Carrier_Fail) as Carrier_Fail,
sum(nostock) as NoStock,
sum(undelivered) as undelivered,
sum(total) as Total

from d2d_pivot where country_desc is not null and Due_for_Delivery = 'Include' 

group by

Due_for_Delivery,
country_desc


order by total desc